In [3]:
import sys
sys.path.append("..")

import dill
from buildmodel.clean_data import drop_null_labels
from buildmodel.get_sample_data import get_data_from_mongo
from pipeline.meetup_pipeline import interest, visibility, meetup_union, random_forest_model, poisson_model
from pipeline.meetup_pipes import PoissonRegression
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from pymongo import MongoClient
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pipeline.meetup_pipeline
from sklearn.preprocessing import LabelBinarizer
from glm.glm import GLM
from evaluation import cv_score
import matplotlib.pyplot as plt

%matplotlib inline


/home/cb/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [1]:
mc = MongoClient()
db = mc['meetups']
events = db['past_3years_meetup_events']
groups = db['groups']
ed = db['events_data']

NameError: name 'MongoClient' is not defined

In [5]:
shard = get_data_from_mongo(0.5)

In [6]:
df = pd.DataFrame(shard)

In [4]:
df =pd.read_csv('../../data/data.gzip', compression='gzip')
df = drop_null_labels(df)

/home/cb/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


poisson regresor

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, interest.fit_transform(df))

lambda = 1

In [7]:
cv_score(poisson_model, X_train.reset_index(drop=True), y_train, metric=mean_squared_error, random_state=1969)

[0.6534648682821419, 0.663837984400206, 0.6668547489314338]

lambda = 5

In [5]:
cv_score(poisson_model, X_train.reset_index(drop=True), y_train, metric=mean_squared_error, random_state=1969)

[0.661743643742077, 0.6623795510689884, 0.6600434101199945]

lambda = 2

In [4]:
cv_score(poisson_model, X_train.reset_index(drop=True), y_train, metric=mean_squared_error, random_state=1969)

[0.6610188748428297, 0.6677384665681728, 0.6559510227039406]

lambda = 0.5

In [4]:
cv_score(poisson_model, X_train.reset_index(drop=True), y_train, metric=mean_squared_error, random_state=1969)

[0.6578632593119466, 0.666550583264152, 0.6634745861031776]

lambda = 0.75

In [4]:
cv_score(poisson_model, X_train.reset_index(drop=True), y_train, metric=mean_squared_error, random_state=1969)

[0.6620525702648565, 0.6608418305831532, 0.6671153029503085]

random forest regressor

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df, interest.fit_transform(df))

In [6]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.46544262349929916, 0.4700739058832407, 0.4813534988509485]

setting n_estimators to 100

In [44]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.45593171513254777, 0.45008682791072974, 0.44635279913599674]

setting minimum samples per leaf to 5

In [4]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.45555444955605207, 0.4487732740212827, 0.4439896503330954]

setting minimum samples per leaf to 10

In [5]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.4648983456180822, 0.46324988612677703, 0.45875686815014827]

setting minimum samples per leaf to 3

In [4]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.44331060187125826, 0.44463084682778004, 0.4500413123183686]

setting minimum samples per leaf to 4

In [4]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.44215055912998324, 0.44681165739421463, 0.4406296890023344]

setting max depth to 10

In [4]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.5979035169134654, 0.6014114235477114, 0.5957360732484588]

setting max depth to 20

In [4]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.5090555017185592, 0.5032393058525156, 0.5032346945024487]

setting max depth to 40

In [5]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.4593121003875722, 0.45531313597683276, 0.4594220180482772]

setting max depth to 50

In [6]:
cv_score(random_forest_model, X_train, y_train, metric=mean_squared_error)

[0.45281373804220426, 0.4517091171756154, 0.45027467839734453]

determine range of people that might attend

In [5]:
random_forest_model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('meetup_features', FeatureUnion(n_jobs=1,
       transformer_list=[('visibility', Pipeline(memory=None,
     steps=[('select_visibility', FeatureSelector(as_numpy=False, key='visibility')), ('binarize', CustomBinarizer())])), ('name_tfidf', Pipeline(memory=None,
     steps=[('select_name', F...n_estimators=100, n_jobs=1,
      oob_score=False, random_state=1969, verbose=0, warm_start=False))])

In [43]:
random_forest_model.named_steps['model'].get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 1969,
 'verbose': 0,
 'warm_start': False}

In [18]:
draw = meetup_union.transform(X_test.iloc[[1]])

In [19]:
trees = random_forest_model.named_steps['model'].estimators_
predictions = np.zeros(len(trees))
for i, tree in enumerate(trees):
    predictions[i] += (tree.predict(draw))

In [37]:
np.e**(np.sort(predictions)[int(len(predictions)*0.05)]) + 1

3.0

In [38]:
np.e**(np.sort(predictions)[int(len(predictions)*0.95)]) + 1

6.999999999999999

In [34]:
np.e**(random_forest_model.predict(X_test.iloc[[1]]))+1

array([3.37042804])

In [42]:
type(round(5.6))

int

In [17]:
y_test

array([0.69314718, 0.        , 1.38629436, ..., 2.07944154, 0.69314718,
       1.38629436])

In [33]:
tfidf_desc = TfidfVectorizer(stop_words='english',max_df=0.99, min_df=.01)

In [34]:
train_desc_vec = tfidf_desc.fit_transform(X_train['plain_text_no_images_description'])
test_desc_vec = tfidf_desc.transform(X_test['plain_text_no_images_description'])

In [35]:
tfidf_name = TfidfVectorizer(stop_words='english', max_df=0.99, min_df=.01)

In [36]:
train_name_vec = tfidf_name.fit_transform(X_train['name'])
test_name_vec = tfidf_name.transform(X_test['name'])

In [152]:
tfidf_name.get_feature_names()

['group']